# Model Assessment of Chris H's 4/24/2022 XGBoost Model

## Imports

In [189]:
%load_ext autoreload
%autoreload 2

import pickle
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from my_functions import plot_confusion_matrix, cost_scorer, savings_scorer, plotly_roc, create_cost_savings, plot_costs_by_threshold, plot_savings_by_threshold, quick_report

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, f1_score, auc, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Functions and Column Lists

In [110]:
def create_groups(grp, df, offset_grp = None):
    df['full_derate'] = False
    
    if offset_grp is not None:
        df = df[df[f'use_for_first_tow_hours_id_group_{offset_grp}'] == False]
        
    df['full_derate'] = (
        np.where(df[f'hours_id_group_{grp}']\
                 .isin(list(df[f'hours_id_group_{grp}']\
                            .loc[(df[f'use_for_first_tow_hours_id_group_{grp}'] == True)])) &
                 (df[f'use_for_first_tow_hours_id_group_{grp}'] == True),
                 True,
                 False)
        )
    
    
    mask = df.loc[(df[f'hours_id_group_{grp}']\
                    .isin(list(df[f'hours_id_group_{grp}']\
                               .loc[(df[f'use_for_first_tow_hours_id_group_{grp}'] == True)]))) &
                    (df['full_derate'] == False)]
    
    row_ls = [i for i in df.index if i not in mask.index]
    
    df = df.loc[row_ls]
    
    grp_col = f'hours_id_group_{grp}'
    
    return df, grp_col

def log_me(col):
    bigg[col] = (
        np.where((bigg[col].notnull()) & 
                 (np.log(bigg[col]) != -np.inf) &
                 (np.log(bigg[col]) != np.inf), 
                 np.log(bigg[col]), np.nan)
    )
    
    return bigg[col]

In [111]:
cat_var = [
    'common_derate_lamp_status',
    'ecu_model',
    'ecu_make',
    'error_category',
    'month',
    'maintenance_before',
    'fuel_temperature_over_32',
    'accelerator_pedal_over_0',
    'cruise_control_set_speed_under_66',
    'switched_battery_voltage_less_than_3276.75',
    'event_description_SCR_related']

normalizers = ['ltd_distance',
             'ltd_engine_time',
             'ltd_fuel']

var = ['barometric_pressure',
       'ltd_distance',
       'engine_coolant_temperature',
       'engine_load',
       'engine_oil_pressure',
       'engine_oil_temperature',
       'engine_rpm',
       'ltd_engine_time',
       'fuel_level',
       'ltd_fuel',
       'fuel_rate',
       'intake_manifold_temperature',
       'speed',
       'turbo_boost_pressure',
       'error_duration_(minutes)'
      ]

log_cols = ['engine_oil_temperature',
            'ltd_engine_time',
            'fuel_rate',
            'turbo_boost_pressure',
            'speed',
            'error_duration_(minutes)']

## Prepare Data

In [112]:
bigg = pd.read_csv('../data/CJH_big_G_042322_420PM.csv', low_memory = False)

In [113]:
bigg['event_time_stamp'] = pd.to_datetime(bigg['event_time_stamp'])

bigg = bigg.loc[(bigg['event_time_stamp'].dt.year > 2014) & (bigg['event_time_stamp'].dt.year <= 2020)]

In [114]:
bigg, grp_col = create_groups('48H', bigg, '1H')

<ipython-input-110-138dc38fa4d7>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [115]:
bigg['common_derate_lamp_status'] = (
    np.where((bigg['lamp_status'] == 22527) | (bigg['lamp_status'] == 18431), True, False)
)

bigg['fuel_temperature_over_32'] = (
    np.where(bigg['fuel_temperature'] > 32, True, False)
)

bigg['accelerator_pedal_over_0'] = (
    np.where(bigg['accelerator_pedal'] > 0, True, False)
)

bigg['cruise_control_set_speed_under_66'] = (
    np.where(bigg['cruise_control_set_speed'] < 66, True, False)
)

bigg['switched_battery_voltage_less_than_3276.75'] = (
    np.where(bigg['switched_battery_voltage'] < 3276.75, True, False)

)

bigg['event_description_SCR_related'] = (
    np.where(bigg['event_description'].str.lower().str.contains('catalyst|aftertreatment|nox|derate'), True, False)
)

In [116]:
ohe_col_ls = []
for i in cat_var:
    ohe = OneHotEncoder()
    transformed = ohe.fit_transform(bigg[[i]])
    bigg[ohe.get_feature_names_out()] = transformed.toarray()
    ohe_col_ls += [i for i in ohe.get_feature_names_out()]

In [117]:
bigg['barometric_pressure'] = np.exp(bigg['barometric_pressure'])

for i in log_cols:
    bigg[i] = log_me(i)

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

invalid value encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

invalid value encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

invalid value encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log

C:\

In [118]:
# def normalize_me(col, normalizer):
#     if col != normalizer:
#         bigg[col] = (
#             np.where((bigg[col].notnull()) & 
#                      (bigg[col]/bigg[normalizer] != -np.inf) &
#                      (bigg[col]/bigg[normalizer] != np.inf), 
#                      bigg[col]/bigg[normalizer] * 100000, 
#                      np.nan))
    
#     return bigg[col]

# normalized_var_ls = []

# for i in var:
#     for j in normalizers:
#         col_name = f'{i}_{j}'
#         bigg[col_name] = normalize_me(i, j)
#         normalized_var_ls.append(col_name)

In [119]:
cat_vars = bigg.groupby(grp_col)[ohe_col_ls].sum().reset_index()

In [120]:
mean_vars = bigg.groupby(grp_col)[var].mean().reset_index().fillna(0)
std_vars = bigg.groupby(grp_col)[var].std().reset_index().fillna(0)

In [121]:
target_values = bigg[[grp_col, 'full_derate']].drop_duplicates()

In [122]:
all_vars = pd.merge(mean_vars, std_vars, on = grp_col).merge(cat_vars, on = grp_col)

In [123]:
X = all_vars.drop(columns = grp_col)
y = target_values['full_derate']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.30, 
                                                    train_size = 0.70, 
                                                    random_state = 111, 
                                                    stratify = y)

## Read In Model and Do Basic Checks

In [124]:
with open('../data/boost_42422.pkl', 'rb') as file:
    pipe = pickle.load(file)

In [125]:
print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00     35587
        True       0.50      0.30      0.37        98

    accuracy                           1.00     35685
   macro avg       0.75      0.65      0.69     35685
weighted avg       1.00      1.00      1.00     35685



In [126]:
print(confusion_matrix(y_test, pipe.predict(X_test)))

[[35558    29]
 [   69    29]]


In [140]:
importances = pd.DataFrame({
    'variable': X_train.columns,
    'importance': pipe['boost'].feature_importances_
})

importances.sort_values('importance', ascending = False).head(15)

,variable,importance
69,error_category_SCR-Related,0.462671
87,accelerator_pedal_over_0_True,0.063697
85,fuel_temperature_over_32_True,0.039064
31,common_derate_lamp_status_True,0.034841
66,ecu_make_Unknown,0.032636
91,switched_battery_voltage_less_than_3276.75_True,0.027404
46,ecu_model_EC60-adv,0.025020
1,ltd_distance_x,0.023595
83,maintenance_before_True,0.021423
7,ltd_engine_time_x,0.019931


## Model Assessment

In [128]:
y_train_prob = pipe.predict_proba(X_train)
y_test_prob = pipe.predict_proba(X_test)

In [129]:
plotly_roc(y_test, y_test_prob)

The optimum tpr vs. fpr threshold value is: 0.0001378673


In [144]:
train_savings_df = create_cost_savings(y_train, y_train_prob, type = 'savings')
test_savings_df = create_cost_savings(y_test, y_test_prob, type = 'savings')

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\jrior\anaconda3\lib\site-pa

In [145]:
train_savings_df.nlargest(5, 'total_savings')

,threshold,total_savings,f1_score,precision,recall
21,0.4,173000.0,0.232945,0.165094,0.395480
23,0.6,171000.0,0.251748,0.214286,0.305085
22,0.5,169000.0,0.239216,0.183183,0.344633
24,0.7,142000.0,0.236620,0.235955,0.237288
20,0.3,141500.0,0.207447,0.135652,0.440678


In [132]:
plot_savings_by_threshold(train_savings_df)

In [133]:
plot_savings_by_threshold(test_savings_df)

In [134]:
quick_report(y_test, y_test_prob, threshold=0.07)

confusion matrix:
 [[35388   199]
 [   44    54]]
False Positives:  199 
False Negatives:  44
f1 score:  0.30769230769230765
precision score:  0.2134387351778656
recall score:  0.5510204081632653
Total Costs:  $319,500.00
Total Savings:  $170,500.00


## Test Model on Different Offsets

In [147]:
# Read in data

bigg = pd.read_csv('../data/CJH_big_G_042322_420PM.csv', low_memory = False)

# Convert to datetime and filter to relevant years

bigg['event_time_stamp'] = pd.to_datetime(bigg['event_time_stamp'])

bigg = bigg.loc[(bigg['event_time_stamp'].dt.year > 2014) & (bigg['event_time_stamp'].dt.year <= 2020)]

In [148]:
def prep_model_data(offset_grp = None):

    # Create groups

    bigg_new, grp_col = create_groups('48H', bigg, offset_grp = offset_grp)

    # Create some columns

    bigg_new['common_derate_lamp_status'] = (
        np.where((bigg_new['lamp_status'] == 22527) | (bigg_new['lamp_status'] == 18431), True, False)
    )

    bigg_new['fuel_temperature_over_32'] = (
        np.where(bigg_new['fuel_temperature'] > 32, True, False)
    )

    bigg_new['accelerator_pedal_over_0'] = (
        np.where(bigg_new['accelerator_pedal'] > 0, True, False)
    )

    bigg_new['cruise_control_set_speed_under_66'] = (
        np.where(bigg_new['cruise_control_set_speed'] < 66, True, False)
    )

    bigg_new['switched_battery_voltage_less_than_3276.75'] = (
        np.where(bigg_new['switched_battery_voltage'] < 3276.75, True, False)

    )

    bigg_new['event_description_SCR_related'] = (
        np.where(bigg_new['event_description'].str.lower().str.contains('catalyst|aftertreatment|nox|derate'), True, False)
    )

    # Get list of OHE columns

    ohe_col_ls = []
    for i in cat_var:
        ohe = OneHotEncoder()
        transformed = ohe.fit_transform(bigg_new[[i]])
        bigg_new[ohe.get_feature_names_out()] = transformed.toarray()
        ohe_col_ls += [i for i in ohe.get_feature_names_out()]

    # Log/Exp-transform skewed columns

    bigg_new['barometric_pressure'] = np.exp(bigg_new['barometric_pressure'])

    for i in log_cols:
        bigg_new[i] = log_me(i)

    # Create aggregated variables

    cat_vars = bigg_new.groupby(grp_col)[ohe_col_ls].sum().reset_index()
    mean_vars = bigg_new.groupby(grp_col)[var].mean().reset_index().fillna(0)
    std_vars = bigg_new.groupby(grp_col)[var].std().reset_index().fillna(0)
    target_values = bigg_new[[grp_col, 'full_derate']].drop_duplicates()
    all_vars = pd.merge(mean_vars, std_vars, on = grp_col).merge(cat_vars, on = grp_col)

    # train-test split

    X = all_vars.drop(columns = grp_col)
    y = target_values['full_derate']

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size = 0.30, 
                                                        train_size = 0.70, 
                                                        random_state = 111, 
                                                        stratify = y)

    return X_train, X_test, y_train, y_test

In [190]:
offsets = [None, '15min', '30min', '45min', '1H', '2H', '3H', '6H', '8H', '11H', '14H']

selected_thresholds = []
test_savings = []
windows_dict = {'costs': {'train': dict(), 'test': dict()}, 
                'savings': {'train': dict(), 'test': dict()}}
probs_dict = {'train': dict(), 'test': dict()}
true_dict = {'train': dict(), 'test': dict()}

for offset in tqdm(offsets):
    X_train, X_test, y_train, y_test = prep_model_data(offset_grp=offset)

    y_train_prob = pipe.predict_proba(X_train)
    y_test_prob = pipe.predict_proba(X_test)

    train_savings_df = create_cost_savings(y_train, y_train_prob, type = 'savings')
    test_savings_df = create_cost_savings(y_test, y_test_prob, type = 'savings')

    train_cost_df = create_cost_savings(y_train, y_train_prob, type = 'costs')
    test_cost_df = create_cost_savings(y_test, y_test_prob, type = 'costs')

    train_cost_df['threshold'] = train_cost_df['threshold'].round(4)
    test_cost_df['threshold'] = test_cost_df['threshold'].round(4)
    train_cost_df['percent_of_savings'] = 100*train_cost_df['total_costs']/train_cost_df.loc[train_cost_df['threshold'] == 1, 'total_costs'].values
    test_cost_df['percent_of_savings'] = 100*test_cost_df['total_costs']/test_cost_df.loc[test_cost_df['threshold'] == 1, 'total_costs'].values

    best_thresh = train_savings_df.nlargest(1, 'total_savings')['threshold'].values[0]
    selected_thresholds.append(best_thresh)
    test_savings.append(test_savings_df.loc[test_savings_df['threshold'] == best_thresh, 'total_savings'].values[0])

    true_dict['train'][offset] = y_train
    true_dict['test'][offset] = y_test

    probs_dict['train'][offset] = y_train_prob
    probs_dict['test'][offset] = y_test_prob

    windows_dict['costs']['train'][offset] = train_cost_df
    windows_dict['costs']['test'][offset] = test_cost_df
    windows_dict['savings']['train'][offset] = train_savings_df
    windows_dict['savings']['test'][offset] = test_savings_df

  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

invalid value encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

invalid value encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

invalid value encountered in log

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:


In [198]:
with open('../data/xgboost_assessment/windows_dict.pickle', 'wb') as file:
    pickle.dump(windows_dict, file, protocol=-1)

with open('../data/xgboost_assessment/probs_dict.pickle', 'wb') as file:
    pickle.dump(probs_dict, file, protocol=-1)

with open('../data/xgboost_assessment/true_dict.pickle', 'wb') as file:
    pickle.dump(true_dict, file, protocol=-1)

with open('../data/xgboost_assessment/selected_thresholds_list.pickle', 'wb') as file:
    pickle.dump(selected_thresholds, file, protocol=-1)

with open('../data/xgboost_assessment/test_savings_list.pickle', 'wb') as file:
    pickle.dump(test_savings, file, protocol=-1)

In [215]:
offsets_readable = ['0min']+offsets[1:]
windows_df = pd.DataFrame({
        'window': offsets_readable,
        'test_savings': test_savings,
        'precision': [windows_dict['savings']['test'][x].nlargest(1, 'total_savings')['precision'].values[0] for x in offsets],
        'recall': [windows_dict['savings']['test'][x].nlargest(1, 'total_savings')['recall'].values[0] for x in offsets],
        'percent_of_actual_costs': [windows_dict['costs']['test'][x].nsmallest(1, 'total_costs')['percent_of_savings'].values[0] for x in offsets]
})

windows_df

,window,test_savings,precision,recall,percent_of_actual_costs
0,0min,459500.0,0.389286,0.660606,44.303030
1,15min,191000.0,0.324841,0.467890,62.935780
2,30min,168500.0,0.242574,0.475728,67.281553
3,45min,159500.0,0.245989,0.442308,69.326923
4,1H,114000.0,0.234899,0.357143,75.918367
5,2H,124500.0,0.234940,0.423913,71.413043
6,3H,145500.0,0.220588,0.511364,66.931818
7,6H,54500.0,0.207921,0.276316,82.894737
8,8H,62000.0,0.157895,0.385714,82.000000
9,11H,62000.0,0.208333,0.307692,80.923077


In [249]:
plotly_roc(true_dict['test']['3H'], 
           probs_dict['test']['3H']
           )

The optimum tpr vs. fpr threshold value is: 0.0012985094


In [222]:

px.line(x = windows_df['window'],
        y = windows_df['test_savings'],
        hover_data=[windows_df['precision'], windows_df['recall'], windows_df['percent_of_actual_costs']],
        labels = {'x':'Window from Derate',
                  'y':'Total Savings',
                  'hover_data_0':'Precision',
                  'hover_data_1':'Recall',
                  'hover_data_2':'Percent of Actual Costs'},
        title = "Our XGBoost model performs well in urgent situations and 3 hours out.<br>This model would have saved you 1/3 of your derate costs.",
        width = 700,
        height = 500)